In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
# class Net(nn.Module): # Architecture: S5
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 2, 3, padding=1)
#         self.conv2 = nn.Conv2d(2, 4, 3, padding=1)
#         self.pool1 = nn.MaxPool2d(2, 2)
#         self.conv3 = nn.Conv2d(4, 8, 3, padding=1)
#         self.conv4 = nn.Conv2d(8, 16, 3, padding=1)

#         self.pool2 = nn.MaxPool2d(2, 2)
#         self.conv5 = nn.Conv2d(16, 32, 3, padding=1)
#         self.conv6 = nn.Conv2d(32, 64, 3, padding=1)
#         self.conv7 = nn.Conv2d(64, 128, 3, padding=1)

#         self.fc1 = nn.Linear(6272, 50)
#         self.fc2 = nn.Linear(50, 10)

#     def forward(self, x):

#         # Start: 1 x 28 x 28

#         x = F.relu(self.conv1(x)) #   2 x 28 x 28
#         x = F.relu(self.conv2(x)) #   4 x 28 x 28
#         x = self.pool1(x)         #   4 x 14 x 14
#         x = F.relu(self.conv3(x)) #   8 x 14 x 14
#         x = F.relu(self.conv4(x)) #  16 x 14 x 14
#         x = self.pool2(x)         #  16 x 7  x 7
#         x = F.relu(self.conv5(x)) #  32 x 7  x 7
#         x = F.relu(self.conv6(x)) #  64 x 7  x 7
#         x = F.relu(self.conv7(x)) # 128 x 7  x 7

#         x = x.view(-1, 6272)
#         x = self.fc1(x)
#         x = F.relu(x)
#         x = self.fc2(x)
#         x = F.log_softmax(x, dim=1)

#         return x


In [3]:
# class Net(nn.Module): # Adding GAP + Batch Norm + Dropout
#     def __init__(self):
#         super(Net, self).__init__()

#         # Convolution blocks with BatchNorm
#         self.conv1 = nn.Conv2d(1, 2, 3, padding=1)
#         self.bn1   = nn.BatchNorm2d(2)

#         self.conv2 = nn.Conv2d(2, 4, 3, padding=1)
#         self.bn2   = nn.BatchNorm2d(4)
#         self.pool1 = nn.MaxPool2d(2, 2)

#         self.conv3 = nn.Conv2d(4, 8, 3, padding=1)
#         self.bn3   = nn.BatchNorm2d(8)

#         self.conv4 = nn.Conv2d(8, 16, 3, padding=1)
#         self.bn4   = nn.BatchNorm2d(16)
#         self.pool2 = nn.MaxPool2d(2, 2)

#         self.conv5 = nn.Conv2d(16, 32, 3, padding=1)
#         self.bn5   = nn.BatchNorm2d(32)

#         self.conv6 = nn.Conv2d(32, 64, 3, padding=1)
#         self.bn6   = nn.BatchNorm2d(64)

#         self.conv7 = nn.Conv2d(64, 128, 3, padding=1)
#         self.bn7   = nn.BatchNorm2d(128)

#         # Global Average Pooling
#         self.gap = nn.AdaptiveAvgPool2d(1)   # -> [batch, 128, 1, 1]

#         # Fully connected (much smaller now)
#         self.fc = nn.Linear(128, 10)

#         self.dropout = nn.Dropout(0.5)

#     def forward(self, x):
#         # Start: 1 x 28 x 28
#         x = F.relu(self.bn1(self.conv1(x)))   # 2x28x28
#         x = F.relu(self.bn2(self.conv2(x)))   # 4x28x28
#         x = self.pool1(x)                     # 4x14x14

#         x = F.relu(self.bn3(self.conv3(x)))   # 8x14x14
#         x = F.relu(self.bn4(self.conv4(x)))   # 16x14x14
#         x = self.pool2(x)                     # 16x7x7

#         x = F.relu(self.bn5(self.conv5(x)))   # 32x7x7
#         x = F.relu(self.bn6(self.conv6(x)))   # 64x7x7
#         x = F.relu(self.bn7(self.conv7(x)))   # 128x7x7

#         # Global Average Pooling
#         x = self.gap(x)                       # 128x1x1
#         x = x.view(x.size(0), -1)             # flatten -> 128

#         x = self.dropout(F.relu(self.fc(x)))  # 128 -> 10
#         return F.log_softmax(x, dim=1)


In [4]:
# class NetBottleneck(nn.Module): # Adding 1x1 kernels
#     def __init__(self):
#         super(NetBottleneck, self).__init__()

#         # Convolution blocks with BatchNorm
#         self.conv1 = nn.Conv2d(1, 2, 3, padding=1)
#         self.bn1   = nn.BatchNorm2d(2)

#         self.conv2 = nn.Conv2d(2, 4, 3, padding=1)
#         self.bn2   = nn.BatchNorm2d(4)
#         self.pool1 = nn.MaxPool2d(2, 2)

#         self.conv3 = nn.Conv2d(4, 8, 3, padding=1)
#         self.bn3   = nn.BatchNorm2d(8)

#         self.conv4 = nn.Conv2d(8, 16, 3, padding=1)
#         self.bn4   = nn.BatchNorm2d(16)
#         self.pool2 = nn.MaxPool2d(2, 2)

#         self.conv5 = nn.Conv2d(16, 32, 3, padding=1)
#         self.bn5   = nn.BatchNorm2d(32)

#         self.conv6 = nn.Conv2d(32, 64, 3, padding=1)
#         self.bn6   = nn.BatchNorm2d(64)

#         # 1x1 conv bottleneck to reduce channels: 64 -> 32
#         self.conv7_1x1 = nn.Conv2d(64, 32, 1)
#         self.bn7_1x1   = nn.BatchNorm2d(32)

#         # final 3x3 conv after bottleneck: 32 -> 64
#         self.conv7 = nn.Conv2d(32, 64, 3, padding=1)
#         self.bn7   = nn.BatchNorm2d(64)

#         # Global Average Pooling
#         self.gap = nn.AdaptiveAvgPool2d(1)   # -> [batch, 64, 1, 1]

#         # Fully connected
#         self.fc = nn.Linear(64, 10)
#         self.dropout = nn.Dropout(0.5)

#     def forward(self, x):
#         # Input: [batch, 1, 28, 28]
#         x = F.relu(self.bn1(self.conv1(x)))   # [batch, 2, 28, 28]
#         x = F.relu(self.bn2(self.conv2(x)))   # [batch, 4, 28, 28]
#         x = self.pool1(x)                     # [batch, 4, 14, 14]

#         x = F.relu(self.bn3(self.conv3(x)))   # [batch, 8, 14, 14]
#         x = F.relu(self.bn4(self.conv4(x)))   # [batch, 16, 14, 14]
#         x = self.pool2(x)                     # [batch, 16, 7, 7]

#         x = F.relu(self.bn5(self.conv5(x)))   # [batch, 32, 7, 7]
#         x = F.relu(self.bn6(self.conv6(x)))   # [batch, 64, 7, 7]

#         # 1x1 bottleneck
#         x = F.relu(self.bn7_1x1(self.conv7_1x1(x)))  # [batch, 32, 7, 7]

#         # final conv
#         x = F.relu(self.bn7(self.conv7(x)))          # [batch, 64, 7, 7]

#         # Global Average Pooling
#         x = self.gap(x)                              # [batch, 64, 1, 1]
#         x = x.view(x.size(0), -1)                    # flatten -> [batch, 64]

#         x = self.dropout(F.relu(self.fc(x)))         # [batch, 10]
#         return F.log_softmax(x, dim=1)


In [5]:
# class NetBottleneck(nn.Module): # Experiment 1: Try with small layers
#     def __init__(self):
#         super(NetBottleneck, self).__init__()

#         # Convolution blocks with BatchNorm
#         self.conv1 = nn.Conv2d(1, 2, 3, padding=1)
#         self.bn1   = nn.BatchNorm2d(2)

#         self.conv2 = nn.Conv2d(2, 4, 3, padding=1)
#         self.bn2   = nn.BatchNorm2d(4)
#         self.pool1 = nn.MaxPool2d(2, 2)

#         self.conv3 = nn.Conv2d(4, 8, 3, padding=1)
#         self.bn3   = nn.BatchNorm2d(8)

#         self.conv4 = nn.Conv2d(8, 16, 3, padding=1)
#         self.bn4   = nn.BatchNorm2d(16)
#         self.pool2 = nn.MaxPool2d(2, 2)

#         self.conv5 = nn.Conv2d(16, 32, 3, padding=1)
#         self.bn5   = nn.BatchNorm2d(32)

#         # self.conv6 = nn.Conv2d(32, 64, 3, padding=1)
#         # self.bn6   = nn.BatchNorm2d(64)

#         # 1x1 conv bottleneck to reduce channels: 64 -> 32
#         self.conv7_1x1 = nn.Conv2d(32, 16, 1)
#         self.bn7_1x1   = nn.BatchNorm2d(16)

#         # final 3x3 conv after bottleneck: 32 -> 64
#         self.conv7 = nn.Conv2d(16, 32, 3, padding=1)
#         self.bn7   = nn.BatchNorm2d(32)

#         # Global Average Pooling
#         self.gap = nn.AdaptiveAvgPool2d(1)   # -> [batch, 64, 1, 1]

#         # Fully connected
#         self.fc = nn.Linear(32, 10)
#         self.dropout = nn.Dropout(0.5)

#     def forward(self, x):
#         # Input: [batch, 1, 28, 28]
#         x = F.relu(self.bn1(self.conv1(x)))   # [batch, 2, 28, 28]
#         x = F.relu(self.bn2(self.conv2(x)))   # [batch, 4, 28, 28]
#         x = self.pool1(x)                     # [batch, 4, 14, 14]

#         x = F.relu(self.bn3(self.conv3(x)))   # [batch, 8, 14, 14]
#         x = F.relu(self.bn4(self.conv4(x)))   # [batch, 16, 14, 14]
#         x = self.pool2(x)                     # [batch, 16, 7, 7]

#         x = F.relu(self.bn5(self.conv5(x)))   # [batch, 32, 7, 7]
#         # x = F.relu(self.bn6(self.conv6(x)))   # [batch, 64, 7, 7]

#         # 1x1 bottleneck
#         x = F.relu(self.bn7_1x1(self.conv7_1x1(x)))  # [batch, 16, 7, 7]

#         # final conv
#         x = F.relu(self.bn7(self.conv7(x)))          # [batch, 32, 7, 7]

#         # Global Average Pooling
#         x = self.gap(x)                              # [batch, 32, 1, 1]
#         x = x.view(x.size(0), -1)                    # flatten -> [batch, 64]

#         x = self.dropout(F.relu(self.fc(x)))         # [batch, 10]
#         return F.log_softmax(x, dim=1)


In [6]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class NetFixedSize(nn.Module): # For 4 onwards
#     def __init__(self):
#         super(NetFixedSize, self).__init__()

#         # Conv layers as per your table
#         self.conv1 = nn.Conv2d(1, 4, 3, padding=1)     # 28x28 -> 28x28
#         self.conv2 = nn.Conv2d(4, 8, 3, padding=1)     # 28x28
#         self.conv3 = nn.Conv2d(8, 4, 1)                # 28x28 bottleneck
#         self.conv4 = nn.Conv2d(4, 8, 3, padding=1)     # 28x28
#         self.conv5 = nn.Conv2d(8, 16, 3, padding=1)    # 28x28
#         self.conv6 = nn.Conv2d(16, 8, 1)               # 28x28 bottleneck
#         self.conv7 = nn.Conv2d(8, 16, 3, padding=1)    # 28x28
#         self.conv8 = nn.Conv2d(16, 32, 3, padding=1)   # 28x28
#         self.conv9 = nn.Conv2d(32, 16, 1)              # 28x28 bottleneck
#         self.conv10 = nn.Conv2d(16, 32, 3, padding=1)  # 28x28

#         # BatchNorm layers
#         self.bn1 = nn.BatchNorm2d(4)
#         self.bn2 = nn.BatchNorm2d(8)
#         self.bn3 = nn.BatchNorm2d(4)
#         self.bn4 = nn.BatchNorm2d(8)
#         self.bn5 = nn.BatchNorm2d(16)
#         self.bn6 = nn.BatchNorm2d(8)
#         self.bn7 = nn.BatchNorm2d(16)
#         self.bn8 = nn.BatchNorm2d(32)
#         self.bn9 = nn.BatchNorm2d(16)
#         self.bn10 = nn.BatchNorm2d(32)

#         # Fully connected
#         self.fc = nn.Linear(32, 10)  # flatten all spatial features
#         self.dropout = nn.Dropout(0.5)

#         # Pooling
#         self.pool = nn.MaxPool2d(2, 2)

#         # Global Average Pooling
#         self.gap = nn.AdaptiveAvgPool2d(1)   # -> [batch, 64, 1, 1]

#     def forward(self, x):
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = self.pool(x)
#         x = F.relu(self.bn2(self.conv2(x)))
#         x = self.pool(x)
#         x = F.relu(self.bn3(self.conv3(x)))  # bottleneck
#         x = self.pool(x)
#         x = F.relu(self.bn4(self.conv4(x)))

#         x = F.relu(self.bn5(self.conv5(x)))

#         x = F.relu(self.bn6(self.conv6(x)))  # bottleneck
#         x = F.relu(self.bn7(self.conv7(x)))

#         x = F.relu(self.bn8(self.conv8(x)))
#         x = F.relu(self.bn9(self.conv9(x)))  # bottleneck
#         x = F.relu(self.bn10(self.conv10(x)))


# # Global Average Pooling
#         x = self.gap(x)                              # [batch, 32, 1, 1]
#         x = x.view(x.size(0), -1)                    # flatten -> [batch, 64]


#         x = self.dropout(F.relu(self.fc(x)))
#         return F.log_softmax(x, dim=1)


In [7]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class NetFixedSize(nn.Module):
#     def __init__(self):
#         super(NetFixedSize, self).__init__()

#         # Convolution layers
#         self.conv1 = nn.Conv2d(1, 8, 3, padding=1)    # 28x28
#         self.bn1 = nn.BatchNorm2d(8)

#         self.conv2 = nn.Conv2d(8, 16, 3, padding=1)   # 28x28
#         self.bn2 = nn.BatchNorm2d(16)

#         self.conv3 = nn.Conv2d(16, 8, 1)             # bottleneck 28x28
#         self.bn3 = nn.BatchNorm2d(8)

#         self.conv4 = nn.Conv2d(8, 16, 3, padding=1)  # 28x28
#         self.bn4 = nn.BatchNorm2d(16)

#         self.conv5 = nn.Conv2d(16, 32, 3, padding=1) # 28x28
#         self.bn5 = nn.BatchNorm2d(32)

#         self.conv6 = nn.Conv2d(32, 16, 1)            # bottleneck 28x28
#         self.bn6 = nn.BatchNorm2d(16)

#         self.conv7 = nn.Conv2d(16, 32, 3, padding=1) # 28x28
#         self.bn7 = nn.BatchNorm2d(32)

#         # Pooling
#         self.pool = nn.MaxPool2d(2, 2)

#         # Global Average Pooling
#         self.gap = nn.AdaptiveAvgPool2d(1)   # -> [batch, 32, 1, 1]

#         # Fully connected
#         self.fc = nn.Linear(32, 10)
#         self.dropout = nn.Dropout(0.5)

#     def forward(self, x):
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = self.pool(x)   # 28 -> 14

#         x = F.relu(self.bn2(self.conv2(x)))
#         x = self.pool(x)   # 14 -> 7

#         x = F.relu(self.bn3(self.conv3(x)))  # bottleneck

#         x = F.relu(self.bn4(self.conv4(x)))
#         x = self.pool(x)   # 7 -> 3

#         x = F.relu(self.bn5(self.conv5(x)))
#         x = F.relu(self.bn6(self.conv6(x)))  # bottleneck
#         x = F.relu(self.bn7(self.conv7(x)))

#         # Global Average Pooling
#         x = self.gap(x)               # [batch, 32, 1, 1]
#         x = x.view(x.size(0), -1)     # flatten -> [batch, 32]

#         x = self.dropout(F.relu(self.fc(x)))
#         return F.log_softmax(x, dim=1)


In [8]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class NetArch(nn.Module):
#     def __init__(self):
#         super(NetArch, self).__init__()

#         # Convolution layers
#         self.conv1 = nn.Conv2d(1, 8, 3, padding=1)      # 28x28 -> 28x28
#         self.bn1 = nn.BatchNorm2d(8)

#         self.conv2 = nn.Conv2d(8, 16, 3, padding=1)     # 14x14 -> 14x14
#         self.bn2 = nn.BatchNorm2d(16)

#         self.conv3 = nn.Conv2d(16, 32, 3, padding=1)    # 7x7 -> 7x7
#         self.bn3 = nn.BatchNorm2d(32)

#         self.bottleneck1 = nn.Conv2d(32, 16, 1)         # 3x3 -> 3x3
#         self.bn4 = nn.BatchNorm2d(16)

#         self.conv4 = nn.Conv2d(16, 32, 3, padding=1)    # 3x3 -> 3x3
#         self.bn5 = nn.BatchNorm2d(32)

#         self.bottleneck2 = nn.Conv2d(32, 16, 1)         # 3x3 -> 3x3
#         self.bn6 = nn.BatchNorm2d(16)

#         self.conv5 = nn.Conv2d(16, 32, 3, padding=1)    # 3x3 -> 3x3
#         self.bn7 = nn.BatchNorm2d(32)

#         # Pooling layers
#         self.pool = nn.MaxPool2d(2, 2)

#         # Global Average Pooling
#         self.gap = nn.AdaptiveAvgPool2d(1)  # Output: [batch, 32, 1, 1]

#         # Fully connected
#         self.fc = nn.Linear(32, 10)
#         self.dropout = nn.Dropout(0.5)

#     def forward(self, x):
#         # Layer 1 + Pool
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = self.pool(x)  # 28 -> 14

#         # Layer 2 + Pool
#         x = F.relu(self.bn2(self.conv2(x)))
#         x = self.pool(x)  # 14 -> 7

#         # Layer 3 + Pool
#         x = F.relu(self.bn3(self.conv3(x)))
#         x = self.pool(x)  # 7 -> 3

#         # Bottleneck 1
#         x = F.relu(self.bn4(self.bottleneck1(x)))

#         # Conv4
#         x = F.relu(self.bn5(self.conv4(x)))

#         # Bottleneck 2
#         x = F.relu(self.bn6(self.bottleneck2(x)))

#         # Conv5
#         x = F.relu(self.bn7(self.conv5(x)))

#         # Global Average Pooling
#         x = self.gap(x)                 # [batch, 32, 1, 1]
#         x = x.view(x.size(0), -1)       # flatten -> [batch, 32]

#         # Fully connected
#         x = self.dropout(F.relu(self.fc(x)))
#         return F.log_softmax(x, dim=1)


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NetCustomArch(nn.Module): # OLD MODEL
    def __init__(self):
        super(NetCustomArch, self).__init__()

        # Conv layers
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)    # 28x28 -> 28x28
        self.bn1 = nn.BatchNorm2d(16)

        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)             # 28x28 -> 28x28 (will pool after)
        self.bn2 = nn.BatchNorm2d(16)

        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)   # 7x7 -> 7x7
        self.bn3 = nn.BatchNorm2d(32)

        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)   # 3x3 -> 3x3
        self.bn4 = nn.BatchNorm2d(32)

        self.conv5 = nn.Conv2d(32, 16, 1)             # 3x3 -> 3x3 bottleneck
        self.bn5 = nn.BatchNorm2d(16)

        self.conv6 = nn.Conv2d(16, 16, 3)   # 3x3 -> 3x3
        self.bn6 = nn.BatchNorm2d(16)

        # Pooling
        self.pool = nn.MaxPool2d(2, 2)

        # Global Average Pooling
        self.gap = nn.AdaptiveAvgPool2d(1)  # final output: [batch, 16, 1, 1]

        # Fully connected
        self.fc = nn.Linear(16, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Conv1
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)   # 7x7 -> 3x3

        # Conv2 + Pool
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)   # 28x28 -> 14x14

        # Conv3
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)   # 7x7 -> 3x3

        # Conv4 + Pool
        x = F.relu(self.bn4(self.conv4(x)))
        # x = self.pool(x)   # 7x7 -> 3x3

        # Conv5
        x = F.relu(self.bn5(self.conv5(x)))

        # Conv6
        x = F.relu(self.bn6(self.conv6(x)))

        # Global Average Pooling
        x = self.gap(x)               # [batch, 16, 1, 1]
        x = x.view(x.size(0), -1)     # flatten -> [batch, 16]

        # Fully connected
        # x = self.dropout(F.relu(self.fc(x)))
        # return F.log_softmax(x, dim=1)

        # Fully connected
        x = self.dropout(x)
        # x = F.relu(self.fc(x))
        x = self.fc(x)

        return F.log_softmax(x, dim=1)


In [10]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class NetCustomArch(nn.Module): # OLD 8 MODEL
#     def __init__(self):
#         super(NetCustomArch, self).__init__()

#         # Conv layers
#         self.conv1 = nn.Conv2d(1, 8, 3, padding=1)    # 28x28 -> 28x28
#         self.bn1 = nn.BatchNorm2d(8)

#         self.conv2 = nn.Conv2d(8, 16, 3,  padding=1)             # 28x28 -> 28x28 (will pool after)
#         self.bn2 = nn.BatchNorm2d(16)

#         self.conv3 = nn.Conv2d(16, 16, 3, padding=1)   # 7x7 -> 7x7
#         self.bn3 = nn.BatchNorm2d(16)

#         self.conv4 = nn.Conv2d(16, 32, 3, padding=1)   # 3x3 -> 3x3
#         self.bn4 = nn.BatchNorm2d(32)

#         self.conv5 = nn.Conv2d(32, 32, 3,  padding=1)             # 3x3 -> 3x3 bottleneck
#         self.bn5 = nn.BatchNorm2d(32)

#         self.conv6 = nn.Conv2d(32, 16, 1)   # 3x3 -> 3x3
#         self.bn6 = nn.BatchNorm2d(16)

#         # Pooling
#         self.pool = nn.MaxPool2d(2, 2)

#         # Global Average Pooling
#         self.gap = nn.AdaptiveAvgPool2d(1)  # final output: [batch, 16, 1, 1]

#         # Fully connected
#         self.fc = nn.Linear(16, 10)
#         self.dropout = nn.Dropout(0.5)

#     def forward(self, x):
#         # Conv1
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = self.pool(x)   # 7x7 -> 3x3

#         # Conv2 + Pool
#         x = F.relu(self.bn2(self.conv2(x)))
#         x = self.pool(x)   # 28x28 -> 14x14

#         # Conv3
#         x = F.relu(self.bn3(self.conv3(x)))
#         x = self.pool(x)   # 7x7 -> 3x3

#         # Conv4 + Pool
#         x = F.relu(self.bn4(self.conv4(x)))


#         # Conv5
#         x = F.relu(self.bn5(self.conv5(x)))
#         x = self.pool(x)   # 7x7 -> 3x3

#         # Conv6
#         x = F.relu(self.bn6(self.conv6(x)))

#         # Global Average Pooling
#         x = self.gap(x)               # [batch, 16, 1, 1]
#         x = x.view(x.size(0), -1)     # flatten -> [batch, 16]

#         # Fully connected
#         x = self.dropout(F.relu(self.fc(x)))
#         return F.log_softmax(x, dim=1)


In [11]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class NetCustomArch(nn.Module): # NEW MODEL
#     def __init__(self):
#         super(NetCustomArch, self).__init__()

#         # Conv layers
#         self.conv1 = nn.Conv2d(1, 16, 3, padding=1)    # 28x28 -> 28x28
#         self.bn1 = nn.BatchNorm2d(16)

#         self.conv2 = nn.Conv2d(16, 32, 3, padding=1)             # 28x28 -> 28x28 (will pool after)
#         self.bn2 = nn.BatchNorm2d(32)

#         self.conv3 = nn.Conv2d(32, 32, 3, padding=1)   # 7x7 -> 7x7
#         self.bn3 = nn.BatchNorm2d(32)

#         self.conv4 = nn.Conv2d(32, 16, 1)   # 3x3 -> 3x3
#         self.bn4 = nn.BatchNorm2d(16)

#         self.conv5 = nn.Conv2d(16, 16, 3, padding=1)             # 3x3 -> 3x3 bottleneck
#         self.bn5 = nn.BatchNorm2d(16)

#         self.conv6 = nn.Conv2d(16, 16, 3)   # 3x3 -> 3x3
#         self.bn6 = nn.BatchNorm2d(16)

#         # Pooling
#         self.pool = nn.MaxPool2d(2, 2)

#         # Global Average Pooling
#         self.gap = nn.AdaptiveAvgPool2d(1)  # final output: [batch, 16, 1, 1]

#         # Fully connected
#         self.fc = nn.Linear(16, 10)
#         self.dropout = nn.Dropout(0.5)

#     def forward(self, x):
#         # Conv1
#         x = F.relu(self.bn1(self.conv1(x)))
#         x = self.pool(x)   # 7x7 -> 3x3

#         # Conv2 + Pool
#         x = F.relu(self.bn2(self.conv2(x)))
#         x = self.pool(x)   # 28x28 -> 14x14

#         # Conv3
#         x = F.relu(self.bn3(self.conv3(x)))
#         x = self.pool(x)   # 7x7 -> 3x3

#         # Conv4 + Pool
#         x = F.relu(self.bn4(self.conv4(x)))
#         # x = self.pool(x)   # 7x7 -> 3x3

#         # Conv5
#         x = F.relu(self.bn5(self.conv5(x)))

#         # Conv6
#         x = F.relu(self.bn6(self.conv6(x)))

#         # Global Average Pooling
#         x = self.gap(x)               # [batch, 16, 1, 1]
#         x = x.view(x.size(0), -1)     # flatten -> [batch, 16]

#         # Fully connected
#         x = self.dropout(x)
#         x = F.relu(self.fc(x))
#         return F.log_softmax(x, dim=1)



In [12]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = NetCustomArch().to(device)

summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 16, 14, 14]           2,320
       BatchNorm2d-5           [-1, 16, 14, 14]              32
         MaxPool2d-6             [-1, 16, 7, 7]               0
            Conv2d-7             [-1, 32, 7, 7]           4,640
       BatchNorm2d-8             [-1, 32, 7, 7]              64
         MaxPool2d-9             [-1, 32, 3, 3]               0
           Conv2d-10             [-1, 32, 3, 3]           9,248
      BatchNorm2d-11             [-1, 32, 3, 3]              64
           Conv2d-12             [-1, 16, 3, 3]             528
      BatchNorm2d-13             [-1, 16, 3, 3]              32
           Conv2d-14             [-1, 1

In [13]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [14]:

from tqdm import tqdm

# Data to plot accuracy and loss graphs
train_losses = []
test_losses = []
train_acc = []
test_acc = []

test_incorrect_pred = {'images': [], 'ground_truths': [], 'predicted_vals': []}

def GetCorrectPredCount(pPrediction, pLabels):
  return pPrediction.argmax(dim=1).eq(pLabels).sum().item()

def train(model, device, train_loader, optimizer, criterion):
  model.train()
  pbar = tqdm(train_loader)

  train_loss = 0
  correct = 0
  processed = 0

  for batch_idx, (data, target) in enumerate(pbar):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()

    # Predict
    pred = model(data)

    # Calculate loss
    loss = criterion(pred, target)
    train_loss+=loss.item()

    # Backpropagation
    loss.backward()
    optimizer.step()

    correct += GetCorrectPredCount(pred, target)
    processed += len(data)

    pbar.set_description(desc= f'Train: Loss={loss.item():0.4f} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')

  train_acc.append(100*correct/processed)
  train_losses.append(train_loss/len(train_loader))

def test(model, device, test_loader, criterion):
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)

            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss # changed: reduction='sum'

            correct += GetCorrectPredCount(output, target)


    test_loss /= len(test_loader.dataset)
    test_acc.append(100. * correct / len(test_loader.dataset))
    test_losses.append(test_loss)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [15]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # changed: added this line
model = NetCustomArch().to(device)

# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # changed: LR
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # changed: LR to 0.01
# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)


# optimizer = optim.Adam(
#     model.parameters(),
#     lr=0.001,
#     weight_decay=1e-4,       # L2 regularization (helps avoid overfitting)
#     betas=(0.9, 0.999),      # can be tuned (0.9, 0.98 sometimes better for transformers/seq models)
#     eps=1e-8                 # numerical stability
# )


scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1) # changed: removed verbose=True

# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, factor=0.5)

# for epoch in range(1, 19): # changed # epochs
#     train(model, device, train_loader, optimizer, epoch)
#     test(model, device, test_loader)


criterion = nn.CrossEntropyLoss() # changed: can add reduction='sum'
num_epochs = 20

for epoch in range(1, num_epochs+1):
  print(f'Epoch {epoch}')
  train(model, device, train_loader, optimizer, criterion)
  test(model, device, test_loader, criterion) # changed: train to test_loader
  scheduler.step()


Epoch 1


Train: Loss=0.4579 Batch_id=937 Accuracy=76.20: 100%|██████████| 938/938 [00:20<00:00, 45.02it/s]


Test set: Average loss: 0.0011, Accuracy: 9848/10000 (98.48%)

Epoch 2


Train: Loss=0.4037 Batch_id=937 Accuracy=85.16: 100%|██████████| 938/938 [00:20<00:00, 46.55it/s]


Test set: Average loss: 0.0007, Accuracy: 9877/10000 (98.77%)

Epoch 3


...

Test set: Average loss: 0.0004, Accuracy: 9940/10000 (99.40%)

Epoch 19


Train: Loss=0.3962 Batch_id=937 Accuracy=90.68: 100%|██████████| 938/938 [00:20<00:00, 45.41it/s]


Test set: Average loss: 0.0004, Accuracy: 9939/10000 (99.39%)

Epoch 20


Train: Loss=0.2127 Batch_id=937 Accuracy=90.84: 100%|██████████| 938/938 [00:20<00:00, 45.02it/s]


Test set: Average loss: 0.0004, Accuracy: 9941/10000 (99.41%)



In [16]:
# S5 Stuff:

In [17]:
# from tqdm import tqdm


# def GetCorrectPredCount(pPrediction, pLabels):
#   return pPrediction.argmax(dim=1).eq(pLabels).sum().item()


# def train(model, device, train_loader, optimizer, criterion):
#     model.train()
#     pbar = tqdm(train_loader)
#     correct = 0
#     processed = 0
#     train_loss = 0
#     for batch_idx, (data, target) in enumerate(pbar):
#         data, target = data.to(device), target.to(device)
#         optimizer.zero_grad()
#         output = model(data)
#         loss = F.nll_loss(output, target)
#         loss.backward()
#         optimizer.step()

#     # Predict
#     pred = model(data)

#     # Calculate loss
#     loss = criterion(pred, target)
#     train_loss+=loss.item()

#     # Backpropagation
#     loss.backward()
#     optimizer.step()

#     correct += GetCorrectPredCount(pred, target)
#     processed += len(data)

#     pbar.set_description(
#             desc=f'Train: Loss={loss.item():.4f} Batch_id={batch_idx} Accuracy={100*correct/processed:.2f}'
#         )

#     return train_loss / len(train_loader)


# def test(model, device, test_loader):
#     model.eval()
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
#             pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
#             correct += pred.eq(target.view_as(pred)).sum().item()

#     test_loss /= len(test_loader.dataset)

#     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#         test_loss, correct, len(test_loader.dataset),
#         100. * correct / len(test_loader.dataset)))

In [18]:
class Net(nn.Module): # Architecture: S5 + Experiments
    def __init__(self):
        super(Net, self).__init__()


        self.pool1 = nn.MaxPool2d(2, 2)
        self.pool2 = nn.MaxPool2d(2, 3)
        self.pool3 = nn.MaxPool2d(2, 3)

        # self.conv1 = nn.Conv2d(1, 2, 3, padding=1)
        # self.conv1 = nn.Conv2d(1, 4, 3, padding=1)
        # self.conv2 = nn.Conv2d(4, 8, 3, padding=1)
        # self.conv1 = nn.Conv2d(1, 16, 3, padding=1)


        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # self.conv4 = nn.Conv2d(64, 128, 3, padding=1)

        self.fc1 = nn.Linear(512, 30)
        self.fc2 = nn.Linear(30, 10)

    def forward(self, x):

        # Start: 1 x 28 x 28

        x = F.relu(self.conv1(x)) #   16 x 28 x 28
        x = self.pool1(x)  #   16 x 14 x 14
        x = F.relu(self.conv2(x)) #   32 x 14 x 14
        x = self.pool2(x)  #   32 x 4 x 4

        # x = F.relu(self.conv3(x)) #   64 x 3 x 3
        # x = self.pool3(x)  #   64 x 1 x 1
        # x = F.relu(self.conv4(x)) #  64 x 1 x 1
        # x = self.pool2(x)         #  16 x 7  x 7
        # x = F.relu(self.conv5(x)) #  32 x 7  x 7
        # x = F.relu(self.conv6(x)) #  64 x 7  x 7
        # x = F.relu(self.conv7(x)) # 128 x 7  x 7

        x = x.view(-1, 512)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)

        return x


In [19]:
class Net(nn.Module): # Architecture: S5 + Experiments
    def __init__(self):
        super(Net, self).__init__()


        self.pool1 = nn.MaxPool2d(2, 2)
        self.pool2 = nn.MaxPool2d(2, 3)
        self.pool3 = nn.MaxPool2d(2, 3)

        # self.conv1 = nn.Conv2d(1, 2, 3, padding=1)
        # self.conv1 = nn.Conv2d(1, 4, 3, padding=1)
        # self.conv2 = nn.Conv2d(4, 8, 3, padding=1)
        # self.conv1 = nn.Conv2d(1, 16, 3, padding=1)


        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # self.conv4 = nn.Conv2d(64, 128, 3, padding=1)

        self.fc1 = nn.Linear(512, 30)
        self.fc2 = nn.Linear(30, 10)

    def forward(self, x):

        # Start: 1 x 28 x 28

        x = F.relu(self.conv1(x)) #   16 x 28 x 28
        x = self.pool1(x)  #   16 x 14 x 14
        x = F.relu(self.conv2(x)) #   32 x 14 x 14
        x = self.pool2(x)  #   32 x 4 x 4

        # x = F.relu(self.conv3(x)) #   64 x 3 x 3
        # x = self.pool3(x)  #   64 x 1 x 1
        # x = F.relu(self.conv4(x)) #  64 x 1 x 1
        # x = self.pool2(x)         #  16 x 7  x 7
        # x = F.relu(self.conv5(x)) #  32 x 7  x 7
        # x = F.relu(self.conv6(x)) #  64 x 7  x 7
        # x = F.relu(self.conv7(x)) # 128 x 7  x 7

        x = x.view(-1, 512)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)

        return x
